# Autogen Logging in Okareo

In [1]:
import os
import autogen

# Put your api key in the environment variable OPENAI_API_KEY
config_list = [
    {
        "model": "gpt-4-0125-preview",
        "api_key": os.environ["OPENAI_API_KEY"],
    }
]

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/mason/Library/Application Support/sagemaker/config.yaml


flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


Set up a team of agents that answer user queries about stocks.

In [3]:
import string, random
from okareo.autogen_logger import AutogenLogger

random_string = ''.join(random.choices(string.ascii_letters, k=5))

logger_config = {
    "api_key": os.environ["OKAREO_API_KEY"],
    "tags": ["autogen-groupchat"],
    "group_name": "autogen-stock-chat-example-"+random_string,
}

autogen_logger = AutogenLogger(logger_config)

with autogen_logger:
    representative = autogen.UserProxyAgent(
        name="customer",
    )

    coder = autogen.AssistantAgent(
        name="coder",
        llm_config=gpt4_config,
        system_message="""You are the Coder. Given a user query, write code to retrieve related stock ticker information. Use whatever API necessary to acquire the relevant stock data.
    You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
    If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    """,
        description="Writes code that helps acquire stock data."
    )
    executor = autogen.UserProxyAgent(
        name="executor",
        system_message="Executor. Execute the code written by the Coder and report the result.",
        human_input_mode="NEVER",
        code_execution_config={
            "last_n_messages": 3,
            "work_dir": "stock",
            "use_docker": False,
        },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
        description="Executes code to acquire stock data."
    )
    analyst = autogen.AssistantAgent(
        name="analyst",
        llm_config=gpt4_config,
        system_message="""You are the Analyst. Please read the stock ticker data and answer the user's question accordingly.""",
        description="Summarizes stock data to answer stock-related queries."
    )

    groupchat = autogen.GroupChat(
        agents=[representative, coder, executor, analyst],
        messages=[],
        max_round=10,
        send_introductions=True,
    )
    groupchat_manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

    chat_result_logged = representative.initiate_chat(
        groupchat_manager,
        message="What is the best-performing stock today?", # initial user message to groupchat
        # message="What will the best-performing stock be tomorrow?",
        summary_method="reflection_with_llm",
    )

[Okareo] Started new logging session with context_token: acbe2d4d-1fa5-4a74-84fa-632c7dc44696
[Okareo] Logging data points under group_name 'autogen-stock-chat-example-LiwcG'.
[Okareo] Logging the following events:
[Okareo] - chat_completion
[Okareo] - function_use
[Okareo] - new_agent
[Okareo] - event
[Okareo] New agent registered under model: customer
[Okareo] New agent registered under model: coder
[Okareo] New agent registered under model: executor
[Okareo] New agent registered under model: analyst
[Okareo] New agent registered under model: chat_manager
customer (to chat_manager):

What will the best-performing stock be tomorrow?

--------------------------------------------------------------------------------
[Okareo] New agent registered under model: checking_agent
[Okareo] New agent registered under model: speaker_selection_agent

Next speaker: analyst

analyst (to chat_manager):

Predicting the exact best-performing stock for tomorrow is challenging due to the unpredictable nat